In [3]:
import json


with open("data/raw/poetrydb_poems.json", "r") as f:
    poem_data = json.load(f)

In [ ]:
# get a list of the poem lines
poems_and_info = poem_data['items']
print(type(poems_and_info[0]))
print(list(poems_and_info)[0])
poem_lines = [poem["lines"] for poem in poems_and_info]
# line_counts = [len(poem["lines"]) for poem in poems_and_info]
# line_counts[0:5]

<class 'dict'>
{'title': 'A Song of Autumn', 'author': 'Adam Lindsay Gordon', 'lines': ['‘WHERE shall we go for our garlands glad', 'At the falling of the year,', 'When the burnt-up banks are yellow and sad,', 'When the boughs are yellow and sere?', 'Where are the old ones that once we had,', 'And when are the new ones near?', 'What shall we do for our garlands glad', 'At the falling of the year?’', '‘Child! can I tell where the garlands go?', 'Can I say where the lost leaves veer', 'On the brown-burnt banks, when the wild winds blow,', 'When they drift through the dead-wood drear?', 'Girl! when the garlands of next year glow,', 'You may gather again, my dear—', 'But I go where the last year’s lost leaves go', 'At the falling of the year.’'], 'linecount': '16'}


In [ ]:
import re
from functools import reduce


# pattern to remove any punctuation
punctuation_pattern = r"[^A-Za-z0-9\s]"


cleaned_poem_lines = []


for poem in poem_lines:   # or poems_and_info, depending on your variable name
  


   processed_lines = []
  
   for line in poem:
       # remove punctuation
       line = re.sub(punctuation_pattern, "", line)


       # collapse multiple spaces → one
       line = re.sub(r"\s+", " ", line).strip().lower()


       # skip empty lines
       if line:
           processed_lines.append(line)


   cleaned_poem_lines.append(processed_lines)


# check
cleaned_poem_lines[0:3]


In [ ]:
# get mean words per line and number of lines for each poem
mean_words_per_poem = []
num_lines_per_poem = []

for poem in poem_lines:
    num_lines = len(poem)
    num_lines_per_poem.append(num_lines)

    if num_lines == 0:
        mean_words_per_poem.append(0)
        print("Cleaned poem lyrics was an empty string, inspect what caused this")
        continue

    word_counts = [len(line.split()) for line in poem]
    
    mean_words = sum(word_counts) / num_lines
    mean_words_per_poem.append(mean_words)

In [ ]:
# before you run this you need to run "!pip install pronouncing" in terminal
import pronouncing


rhyme_feature_poems = []


MAX_OFFSET = 4   # how many lines ahead to check for rhyme (captures ABAB, AABB, etc.)


for poem in cleaned_poem_lines:
  
   # grabs the last word of each line in the poem 
   # (lines already cleaned earlier in preprocessing)
   last_words = []
   for line in poem:
       words = line.split()
       last_words.append(words[-1].lower())
  
   # no rhyme density if we only have one line (or none)
   if len(last_words) < 2:
       rhyme_feature_poems.append(0)
       continue


   rhyme_hits = 0       # counts how many pairs rhyme
   rhyme_checks = 0     # total number of pairs checked (with offsets)


   # count rhyme for pairs up to MAX_OFFSET lines apart
   # this captures ABAB, AABB, ABCB, etc. not just adjacent lines
   for i in range(len(last_words)):
       for offset in range(1, MAX_OFFSET + 1):
           if i + offset >= len(last_words):
               break


           w1 = last_words[i]
           w2 = last_words[i + offset]
           rhyme_checks += 1


           # using CMUdict: returns the word pronounced as phonemes
           phones1 = pronouncing.phones_for_word(w1)
           phones2 = pronouncing.phones_for_word(w2)


           if phones1 and phones2:
               # gets all words that rhyme (by phonetic ending) with w1
               rhymes1 = pronouncing.rhymes(w1)
              
               # checks if w2 is in this rhyming phoneme-based list
               if w2 in rhymes1:
                   rhyme_hits += 1
           else:
               # for slang, invented words, missing CMUdict entries, etc.,
               # estimate rhyme manually: check last 2 letters as a weak heuristic
               if w1[-2:] == w2[-2:]:
                   rhyme_hits += 1


   # what percentage of all checked pairs rhyme
   rhyme_density_value = rhyme_hits / rhyme_checks
   rhyme_feature_poems.append(rhyme_density_value)


# inspect example scores
rhyme_feature_poems[:20]


In [ ]:
# checks - check these are the correct shape

print(len(cleaned_poem_lines))
print(len(cleaned_poem_lines) == len(mean_words_per_poem) &
     len(mean_words_per_poem) == len(num_lines_per_poem) & 
     len(num_lines_per_poem) == len(rhyme_feature_poems))

3413
3413
3413


In [5]:
with open("data/processed/combined_songs_large_fixed.json", "r") as f:
        song_data = json.load(f)

In [ ]:
import re
from functools import reduce

songs_and_info = song_data['items']
# one song dictionary
print(songs_and_info[0])
# Pulling each song's lyrics
# Note: each item also contains title, album and duration if we want that later

raw_song_texts = [song["lyrics"] for song in songs_and_info]
raw_song_texts[0:5]


{'title': 'Father Figure', 'artist': 'Taylor Swift', 'spotify_artist_name': 'Taylor Swift', 'album': 'The Life of a Showgirl', 'release_date': '2025-10-03', 'duration_ms': 212777, 'popularity': 94, 'lyrics': '[Verse 1]\nWhen I found you, you were young, wayward, lost in the cold\nPulled up to you in the Jag\', turned your rags into gold\nThe winding road leads to the chateau\n"You remind me of a younger me," I saw potential\n[Chorus]\nI\'ll be your father figure,\n \nI drink that brown liquor\nI can make deals with the devil because my dick\'s bigger\nThis love is pure profit, just step into my office\nI dry your tears with my sleeve\n[Post-Chorus]\nLeave it with me, I protect the family\nLeave it with me, I protect the family\n[Verse 2]\nI pay the check before it kisses the mahogany grain\nSaid, "They wanna see you rise, they don\'t want you to reign"\nI showed you all the tricks of the trade\nAll I asked for is your loyalty, my dear protégé\n[Chorus]\nI\'ll be your father figure,\n I

['[Verse 1]\nWhen I found you, you were young, wayward, lost in the cold\nPulled up to you in the Jag\', turned your rags into gold\nThe winding road leads to the chateau\n"You remind me of a younger me," I saw potential\n[Chorus]\nI\'ll be your father figure,\n \nI drink that brown liquor\nI can make deals with the devil because my dick\'s bigger\nThis love is pure profit, just step into my office\nI dry your tears with my sleeve\n[Post-Chorus]\nLeave it with me, I protect the family\nLeave it with me, I protect the family\n[Verse 2]\nI pay the check before it kisses the mahogany grain\nSaid, "They wanna see you rise, they don\'t want you to reign"\nI showed you all the tricks of the trade\nAll I asked for is your loyalty, my dear protégé\n[Chorus]\nI\'ll be your father figure,\n I drink that brown liquor\nI can make deals with the devil because my dick\'s bigger\nThis love is pure profit, just step into my office\nThey\'ll know your name in the streets\n\n[Post-Chorus]\nLeave it with

In [ ]:
# when there are labels (verse, chorus, etc) 
# in between two enters remove them so we don't count the label 
# as an extra line 
cleaned_songs = [
    re.sub(r"\r?\n\[(Verse \d+|Chorus)\]\r?\n", "\n", song)
    for song in raw_song_texts
]
cleaned_songs[20:100]

['8 Contributors\nTranslations\nไทย (Thai)\nPolski\nsrpski\nElizabeth Taylor (Track by Track) Lyrics\n[Spoken]\nTrack number two on \nThe Life of a Showgirl\n is the song "Elizabeth Taylor"\nElizabeth Taylor\u2005is\u2005one of the\u2005most ultimate, quintessential showgirls I could\u2005ever imagine\nYou know, not in literal sense as much as she was under a microscope so, so intense and she handled it with humor and she got along with her life\nShe continued to make incredible art and so this is a love song kind of through the lens of the motif of what she had to go through in her life and sort of the parallels that I feel in my own life\nYou know, role models are pretty hard to come by, but I would absolutely say that she\'s one of mine.',
 '12 Contributors\nTranslations\nไทย (Thai)\nPolski\nsrpski\nOpalite (Track by Track) Lyrics\n[Spoken]\n"Opalite" is a song on my album that I think is just so infectiously, contagiously happy and it\'s really an expression of happiness\nThe song 

In [ ]:
# remove verse/chorus labels, newline characters and punctuations
replacements_space = [
    r"\[Verse \d+\]",
    r"\[Chorus\]",
    r"\r?\n",
]

punctuation_pattern = r"[^A-Za-z0-9\s]"

cleaned_song_lines = []

for song in cleaned_songs:
    
    lines = song.split("\n")
    processed_lines = []
    
    for line in lines:
        # remove labels and punctuation
        line = reduce(lambda text, pat: re.sub(pat, " ", text), replacements_space, line)
        line = re.sub(punctuation_pattern, "", line)

        # turns multiple spaces into one
        line = re.sub(r"\s+", " ", line).strip().lower()

        # removes any lines that are empty or only contain whitespace
        if line:
            processed_lines.append(line)

    cleaned_song_lines.append(processed_lines)

# check
cleaned_song_lines[0:3]


[['when i found you you were young wayward lost in the cold',
  'pulled up to you in the jag turned your rags into gold',
  'the winding road leads to the chateau',
  'you remind me of a younger me i saw potential',
  'ill be your father figure',
  'i drink that brown liquor',
  'i can make deals with the devil because my dicks bigger',
  'this love is pure profit just step into my office',
  'i dry your tears with my sleeve',
  'postchorus',
  'leave it with me i protect the family',
  'leave it with me i protect the family',
  'i pay the check before it kisses the mahogany grain',
  'said they wanna see you rise they dont want you to reign',
  'i showed you all the tricks of the trade',
  'all i asked for is your loyalty my dear protg',
  'ill be your father figure',
  'i drink that brown liquor',
  'i can make deals with the devil because my dicks bigger',
  'this love is pure profit just step into my office',
  'theyll know your name in the streets',
  'postchorus',
  'leave it wit

In [ ]:
# get mean words per line and number of lines for each song
mean_words_per_song = []
num_lines_per_song = []
total_words_per_song = []

for song in cleaned_song_lines:

    num_lines = len(song)
    num_lines_per_song.append(num_lines)

    if num_lines == 0:
        mean_words_per_song.append(0)
        print("Cleaned songs lyrics was an empty string, inspect what caused this")
        continue

    word_counts = [len(line.split()) for line in song]

    mean_words = sum(word_counts) / num_lines
    mean_words_per_song.append(mean_words)
    total_words_per_song.append(sum(word_counts))


In [ ]:
# calculating mean WPM for each song
durations_ms = [song["duration_ms"] for song in songs_and_info]
durations_min = [song["duration_ms"] / 60000 for song in songs_and_info]

wpm_per_song = []

for words, minutes in zip(total_words_per_song, durations_min):
    if minutes == 0:
        wpm_per_song.append(0)
        print("If we have a song with no time length, this is an error and we set WPM to 0")
    else:
        wpm_per_song.append(words / minutes)

In [23]:
# checks
print(len(cleaned_songs))
print(len(mean_words_per_song))
print(len(num_lines_per_song))
print(len(wpm_per_song))

3000
3000
3000
3000


In [ ]:
import numpy as np

# combining these calculated features into a np array before standardizing 
features_array_poems_unstandardized = np.column_stack([
    mean_words_per_poem,
    num_lines_per_poem
])

features_array_songs_unstandardized = np.column_stack([
    mean_words_per_song,
    num_lines_per_song,
    wpm_per_song
])
print(features_array_poems_unstandardized.shape)
print(features_array_songs_unstandardized.shape)

(3413, 2)
(3000, 3)


In [29]:
# standardizing
poem_means = features_array_poems_unstandardized.mean(axis=0)
poem_stds  = features_array_poems_unstandardized.std(axis=0)

poem_stds[poem_stds == 0] = 1

features_array_poems = (features_array_poems_unstandardized - poem_means) / poem_stds

song_means = features_array_songs_unstandardized.mean(axis=0)
song_stds  = features_array_songs_unstandardized.std(axis=0)

song_stds[song_stds == 0] = 1

features_array_songs = (features_array_songs_unstandardized - song_means) / song_stds